In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import tree
data = pd.read_csv("Populations.csv")
columns = ["Region, subregion, country or area *", "Year", "Type", "Total Population, as of 1 January (thousands)", "Total Population, as of 1 July (thousands)", "Male Population, as of 1 July (thousands)", "Female Population, as of 1 July (thousands)", "Median Age, as of 1 July (years)", "Population Change (thousands)", "Births (thousands)","Total Fertility Rate (live births per woman)", "Population Density, as of 1 July (persons per square km)", "Parent code", "Under-Five Mortality (deaths under age 5 per 1,000 live births)","Mortality before Age 40, both sexes (deaths under age 40 per 1,000 live births)","Net Number of Migrants (thousands)","Live Births Surviving to Age 1 (thousands)","Life Expectancy at Age 80, both sexes (years)","Total Deaths (thousands)","Life Expectancy at Birth, both sexes (years)","Sex Ratio at Birth (males per 100 female births)","Net Reproduction Rate (surviving daughters per woman)","Rate of Natural Change (per 1,000 population)","Population Growth Rate (percentage)","Births by women aged 15 to 19 (thousands)","Crude Birth Rate (births per 1,000 population)"]
data = data[columns]
heatmap_data = data
world_data = data[ data["Type"] == "World"]
region_data = data[data["Type"] == "SDG region"]
data = data[data["Type"] == "Country/Area"]

data = data.dropna()

In [ ]:
depths = [5, 10, 20, 50, 100, 200, 500]
y = data["Total Population, as of 1 January (thousands)"]
X = data.drop(["Parent code", "Region, subregion, country or area *", "Type","Total Population, as of 1 January (thousands)", "Total Population, as of 1 July (thousands)","Male Population, as of 1 July (thousands)", "Female Population, as of 1 July (thousands)"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=476)
X_subtrain, X_val, y_subtrain, y_val = train_test_split(X, y, test_size=0.125, random_state=476)
for depth in depths:
    clf = tree.DecisionTreeRegressor(criterion='squared_error', max_depth=depth)
    clf.fit(X_subtrain, y_subtrain)
    y_predict = clf.predict(X_val)
    print(mean_squared_error(y_val,y_predict, squared=False))